fingerprint, molecular descriptor 노드 따로 해서 처리하다가 합치는 딥러닝 모델 구축

#1. GRU

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GRU, Dropout, Concatenate
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


preprocessing

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/belka/merged_data.csv')
df.columns

Index(['id', 'buildingblock1_smiles', 'buildingblock2_smiles',
       'buildingblock3_smiles', 'molecule_smiles', 'protein_name', 'binds',
       'buildingblock1_AXp-7dv', 'buildingblock2_MATS1i',
       'buildingblock3_AATSC0c', 'buildingblock3_AATSC1c',
       'buildingblock3_AATSC5c', 'molecule_BCUTpe-1l', 'molecule_n11FHRing',
       'molecule_JGI7'],
      dtype='object')

In [ ]:
df.shape

(10000, 15)

In [ ]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds,buildingblock1_AXp-7dv,buildingblock2_MATS1i,buildingblock3_AATSC0c,buildingblock3_AATSC1c,buildingblock3_AATSC5c,molecule_BCUTpe-1l,molecule_n11FHRing,molecule_JGI7
0,235155307,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",HSA,0,0.007479,-0.076329,0.011726,-0.004464,-0.000764,1.213339,0,0.011746
1,69679927,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",HSA,0,0.007422,-0.335938,0.028311,-0.014790,-0.002671,1.213337,0,0.014109
2,168178639,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",HSA,1,0.007064,0.102677,0.055730,-0.039457,0.019188,1.213337,0,0.010305
3,198307966,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",HSA,0,0.007173,0.198399,0.014093,-0.002780,-0.000912,1.213337,0,0.010504
4,91040749,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",HSA,0,0.006740,0.051733,0.014762,-0.006446,-0.003343,1.213335,0,0.013065


In [ ]:
def split_one_hot_columns(df, column_name, prefix):
    one_hot_df = df[column_name].apply(lambda x: pd.Series(eval(x)))
    one_hot_df.columns = [f'{prefix}_{i}' for i in range(one_hot_df.shape[1])]
    return one_hot_df

In [ ]:
# 각 열을 분할하고 원래 데이터프레임과 병합
bb1_df = split_one_hot_columns(df, 'buildingblock1_smiles', 'bb1_smiles')
bb2_df = split_one_hot_columns(df, 'buildingblock2_smiles', 'bb2_smiles')
bb3_df = split_one_hot_columns(df, 'buildingblock3_smiles', 'bb3_smiles')
mol_df = split_one_hot_columns(df, 'molecule_smiles', 'mol_smiles')

In [ ]:
bb1_df.head()

,bb1_smiles_0,bb1_smiles_1,bb1_smiles_2,bb1_smiles_3,bb1_smiles_4,bb1_smiles_5,bb1_smiles_6,bb1_smiles_7,bb1_smiles_8,bb1_smiles_9,...,bb1_smiles_1014,bb1_smiles_1015,bb1_smiles_1016,bb1_smiles_1017,bb1_smiles_1018,bb1_smiles_1019,bb1_smiles_1020,bb1_smiles_1021,bb1_smiles_1022,bb1_smiles_1023
0,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
df.dtypes

id                          int64
buildingblock1_smiles      object
buildingblock2_smiles      object
buildingblock3_smiles      object
molecule_smiles            object
protein_name               object
binds                       int64
buildingblock1_AXp-7dv    float64
buildingblock2_MATS1i     float64
buildingblock3_AATSC0c    float64
buildingblock3_AATSC1c    float64
buildingblock3_AATSC5c    float64
molecule_BCUTpe-1l        float64
molecule_n11FHRing          int64
molecule_JGI7             float64
dtype: object

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Dense, GRU, Dropout, Concatenate, Reshape

# Columns for input features
X1 = pd.concat([bb1_df, bb2_df, bb3_df], axis=1)
X2 = mol_df
X3_columns = ['buildingblock1_AXp-7dv', 'buildingblock2_MATS1i',
              'buildingblock3_AATSC0c', 'buildingblock3_AATSC1c', 'buildingblock3_AATSC5c']
X4_columns = ['molecule_BCUTpe-1l', 'molecule_n11FHRing', 'molecule_JGI7']
y_column = 'binds'

# Split the data into training and testing sets
X1_train, X1_test = train_test_split(X1, test_size=0.2, random_state=42)
X2_train, X2_test = train_test_split(X2, test_size=0.2, random_state=42)
X3_train, X3_test = train_test_split(df[X3_columns], test_size=0.2, random_state=42)
X4_train, X4_test = train_test_split(df[X4_columns], test_size=0.2, random_state=42)
y_train, y_test = train_test_split(df[y_column], test_size=0.2, random_state=42)

In [ ]:
# Build the GRU model
# First input (ECFPs)
input1 = Input(shape=(X1_train.shape[1],))
embedding1 = Dense(128, activation='relu')(input1)
reshaped_embedding1 = Reshape((1, 128))(embedding1)
gru1 = GRU(64, return_sequences=False)(reshaped_embedding1)
dropout1 = Dropout(0.2)(gru1)

# Second input (molecular descriptor)
input2 = Input(shape=(X2_train.shape[1],))
dense2 = Dense(64, activation='relu')(input2)
dropout2 = Dropout(0.2)(dense2)

# Third input
input3 = Input(shape=(X3_train.shape[1],))
dense3 = Dense(64, activation='relu')(input3)
dropout3 = Dropout(0.2)(dense3)

# Fourth input
input4 = Input(shape=(X4_train.shape[1],))
dense4 = Dense(64, activation='relu')(input4)
dropout4 = Dropout(0.2)(dense4)

# Combine the four inputs
merged = Concatenate()([dropout1, dropout2, dropout3, dropout4])
dense5 = Dense(32, activation='relu')(merged)
output = Dense(1, activation='sigmoid')(dense5)

# Define the model
model = Model(inputs=[input1, input2, input3, input4], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 3072)]               0         []                            
                                                                                                  
 dense_6 (Dense)             (None, 128)                  393344    ['input_5[0][0]']             
                                                                                                  
 reshape_1 (Reshape)         (None, 1, 128)               0         ['dense_6[0][0]']             
                                                                                                  
 input_6 (InputLayer)        [(None, 1024)]               0         []                            
                                                                                            

In [ ]:
history = model.fit([X1_train, X2_train, X3_train, X4_train], y_train, epochs=50, batch_size=32, validation_data=([X1_test, X2_test, X3_test, X4_test], y_test))

Epoch 1/50
250/250 [==============================] - 4s 8ms/step - loss: nan - accuracy: 0.9965 - val_loss: nan - val_accuracy: 0.9950
Epoch 2/50
250/250 [==============================] - 1s 6ms/step - loss: nan - accuracy: 0.9965 - val_loss: nan - val_accuracy: 0.9950
Epoch 3/50
250/250 [==============================] - 1s 6ms/step - loss: nan - accuracy: 0.9965 - val_loss: nan - val_accuracy: 0.9950
Epoch 4/50
250/250 [==============================] - 1s 6ms/step - loss: nan - accuracy: 0.9965 - val_loss: nan - val_accuracy: 0.9950
Epoch 5/50
250/250 [==============================] - 1s 6ms/step - loss: nan - accuracy: 0.9965 - val_loss: nan - val_accuracy: 0.9950
Epoch 6/50
250/250 [==============================] - 1s 6ms/step - loss: nan - accuracy: 0.9965 - val_loss: nan - val_accuracy: 0.9950
Epoch 7/50
250/250 [==============================] - 1s 6ms/step - loss: nan - accuracy: 0.9965 - val_loss: nan - val_accuracy: 0.9950
Epoch 8/50
250/250 [============================

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate([X1_test, X2_test, X3_test, X4_test], y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

63/63 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.9950
Test Loss: nan, Test Accuracy: 0.9950000047683716
